In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing

In [2]:
path = '/data/public/MLA/VCHAMPS-Test/demographics_static_test.csv'
demographics_static_test = pd.read_csv(path).iloc[:,1:]
demographics_static_test.head()

,Internalpatientid,Ethnicity,Gender,Races,Veteran flag
0,168807,Not Hispanic or Latino,Male,Black or African American,Yes
1,16884,Not specified,Male,Not specified (no value),Yes
2,168868,Not specified,Male,Not specified (no value),Yes
3,16928,Not Hispanic or Latino,Male,White,Yes
4,17062,Not Hispanic or Latino,Male,White,Yes


In [3]:
demographics_static_test = demographics_static_test[demographics_static_test['Veteran flag'] == 'Yes']

In [4]:
pd.DataFrame(demographics_static_test['Races'].value_counts())

,Races
White,22411
Not specified (no value),5727
Black or African American,4360
Other,1653
White ; Other,375
Asian,119
Black or African American ; White,60
Black or African American ; Other,43
White ; Asian,8
Asian ; Other,6


In [5]:
def races_combine(data):
    for i in range(len(data['Races'])):
        if  data['Races'].iloc[i] == 'White ; Other':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'Black or African American ; Other':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Asian ; Other':
            data['Races'].iloc[i] = 'Asian'
        elif data['Races'].iloc[i] == '(Censored)':
            data['Races'].iloc[i] = 'Other'
        elif data['Races'].iloc[i] == 'Not specified (no value)':
            data['Races'].iloc[i] = 'Other'
        elif data['Races'].iloc[i] == 'Black or African American ; White':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Black or African American ; Asian':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'White ; Asian':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'White ; Asian ; Other':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'Black or African American ; White ; Asian':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Black or African American ; White ; Other':
            data['Races'].iloc[i] = 'Black or African American'
        
    return data



In [6]:
demographics_static_test2 = races_combine(demographics_static_test)
demographics_static_test2['Races'].value_counts()

/tmp/ipykernel_124138/2746446931.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'Other'
/tmp/ipykernel_124138/2746446931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'White'
/tmp/ipykernel_124138/2746446931.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'Black or African American'
/tmp/ipykernel_124138/2746446931.py:6: SettingWithCopyWarning: 
A 

White                        22796
Other                         7381
Black or African American     4466
Asian                          125
Name: Races, dtype: int64

In [7]:
demographics_static_test3 = demographics_static_test2.copy()
demographics_static_test3['Ethnicity'].replace(['Hispanic or Latino','Not Hispanic or Latino','Not specified'],[0,1,2],inplace=True)
demographics_static_test3['Races'].replace(['White','Other','Black or African American', 'Asian'],[0,1,2,3],inplace=True)
demographics_static_test3['Gender'].replace(['Male','Female'],[1,0],inplace=True)

In [8]:
# One hot encoding races and ethnicity
data_dummy = pd.get_dummies(data = demographics_static_test3, columns = ['Ethnicity','Races'])
data_dummy = data_dummy[['Internalpatientid', 
       'Ethnicity_0', 'Ethnicity_1', 'Ethnicity_2', 'Races_0', 'Races_1',
       'Races_2', 'Races_3']]

In [9]:
demographics_static_test3 = pd.concat([demographics_static_test3,data_dummy.iloc[:,1:]],axis = 1)
demographics_static_test3

,Internalpatientid,Ethnicity,Gender,Races,Veteran flag,Ethnicity_0,Ethnicity_1,Ethnicity_2,Races_0,Races_1,Races_2,Races_3
0,168807,1,1,2,Yes,0,1,0,0,0,1,0
1,16884,2,1,1,Yes,0,0,1,0,1,0,0
2,168868,2,1,1,Yes,0,0,1,0,1,0,0
3,16928,1,1,0,Yes,0,1,0,1,0,0,0
4,17062,1,1,0,Yes,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34808,168155,1,1,0,Yes,0,1,0,1,0,0,0
34809,168264,1,1,0,Yes,0,1,0,1,0,0,0
34810,168378,1,1,2,Yes,0,1,0,0,0,1,0
34811,168462,1,1,0,Yes,0,1,0,1,0,0,0


In [11]:
demographics_static_test3 = demographics_static_test3.groupby('Internalpatientid').apply(lambda x:x)
demographics_static_test3 = demographics_static_test3.sort_values('Internalpatientid').reset_index(drop =True)
demographics_static_test3

/tmp/ipykernel_124138/3237507649.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  demographics_static_test3 = demographics_static_test3.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,Ethnicity,Gender,Races,Veteran flag,Ethnicity_0,Ethnicity_1,Ethnicity_2,Races_0,Races_1,Races_2,Races_3
0,6,1,1,0,Yes,0,1,0,1,0,0,0
1,7,1,1,2,Yes,0,1,0,0,0,1,0
2,9,2,1,1,Yes,0,0,1,0,1,0,0
3,12,1,1,0,Yes,0,1,0,1,0,0,0
4,17,2,1,1,Yes,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34763,169037,1,1,0,Yes,0,1,0,1,0,0,0
34764,169045,1,1,0,Yes,0,1,0,1,0,0,0
34765,169058,2,1,1,Yes,0,0,1,0,1,0,0
34766,169059,1,0,1,Yes,0,1,0,0,1,0,0


In [13]:
#demographics_static_test3.to_csv('/home/daisy/FDA_Dataset/demographics_static_test_clean.csv')